In [1]:
import os
import subprocess

# -------------------------------
# Mosaic Tree DSM
# -------------------------------

# Directory containing the tree DSM tiles
tree_dsm_dir = "../data/tiles_output/tree_dsm"
# Paths for the intermediate VRT and final output TIFF for tree DSM
tree_vrt_path = "../data/philly/mosaic_tree_dsm.vrt"
tree_output_path = "../data/philly/mosaic_tree_dsm.tif"

# List all the tree DSM TIFF files in the directory
tree_tif_files = [os.path.join(tree_dsm_dir, f) for f in os.listdir(tree_dsm_dir) if f.endswith(".tif")]

# Build the VRT mosaic without loading data into memory for tree DSM
build_vrt_command = ["gdalbuildvrt", tree_vrt_path] + tree_tif_files
print("\nRunning command:", " ".join(build_vrt_command))
subprocess.run(build_vrt_command, check=True)

# Convert the VRT to a GeoTIFF for tree DSM (this processes the data in chunks)
translate_command = ["gdal_translate", tree_vrt_path, tree_output_path]
print("\nRunning command:", " ".join(translate_command))
subprocess.run(translate_command, check=True)

print("\nTree DSM mosaic created successfully at:", tree_output_path)


# -------------------------------
# Mosaic Building DSM
# -------------------------------

# Directory containing the building DSM tiles
bldg_dsm_dir = "../data/tiles_output/building_dsm"
# Paths for the intermediate VRT and final output TIFF for building DSM
bldg_vrt_path = "../data/philly/mosaic_building_dsm.vrt"
bldg_output_path = "../data/philly/mosaic_building_dsm.tif"

# List all the building DSM TIFF files in the directory
bldg_tif_files = [os.path.join(bldg_dsm_dir, f) for f in os.listdir(bldg_dsm_dir) if f.endswith(".tif")]

# Build the VRT mosaic without loading data into memory for building DSM
build_vrt_command = ["gdalbuildvrt", bldg_vrt_path] + bldg_tif_files
print("\nRunning command:", " ".join(build_vrt_command))
subprocess.run(build_vrt_command, check=True)

# Convert the VRT to a GeoTIFF for building DSM
translate_command = ["gdal_translate", bldg_vrt_path, bldg_output_path]
print("\nRunning command:", " ".join(translate_command))
subprocess.run(translate_command, check=True)

print("\nBuilding DSM mosaic created successfully at:", bldg_output_path)



Running command: gdalbuildvrt ../data/philly/mosaic_tree_dsm.vrt ../data/tiles_output/tree_dsm\ndsm_tile_0_tree_dsm.tif ../data/tiles_output/tree_dsm\ndsm_tile_10_tree_dsm.tif ../data/tiles_output/tree_dsm\ndsm_tile_11_tree_dsm.tif ../data/tiles_output/tree_dsm\ndsm_tile_12_tree_dsm.tif ../data/tiles_output/tree_dsm\ndsm_tile_13_tree_dsm.tif ../data/tiles_output/tree_dsm\ndsm_tile_14_tree_dsm.tif ../data/tiles_output/tree_dsm\ndsm_tile_15_tree_dsm.tif ../data/tiles_output/tree_dsm\ndsm_tile_16_tree_dsm.tif ../data/tiles_output/tree_dsm\ndsm_tile_17_tree_dsm.tif ../data/tiles_output/tree_dsm\ndsm_tile_18_tree_dsm.tif ../data/tiles_output/tree_dsm\ndsm_tile_19_tree_dsm.tif ../data/tiles_output/tree_dsm\ndsm_tile_1_tree_dsm.tif ../data/tiles_output/tree_dsm\ndsm_tile_20_tree_dsm.tif ../data/tiles_output/tree_dsm\ndsm_tile_21_tree_dsm.tif ../data/tiles_output/tree_dsm\ndsm_tile_22_tree_dsm.tif ../data/tiles_output/tree_dsm\ndsm_tile_23_tree_dsm.tif ../data/tiles_output/tree_dsm\ndsm_tile_

In [3]:
import rasterio
import matplotlib.pyplot as plt
import numpy as np

def visualize_and_export(tif_path, png_path, scale=0.1):
    """
    Visualize a large GeoTIFF by downsampling it and export the result as a PNG.
    
    Parameters:
        tif_path (str): Path to the input GeoTIFF.
        png_path (str): Path for the output PNG file.
        scale (float): Factor by which to downsample the image. 
                       0.1 means 10% of the original dimensions.
    """
    with rasterio.open(tif_path) as src:
        # Get original dimensions
        orig_height = src.height
        orig_width = src.width
        print(f"Original shape for {tif_path}: {orig_width}x{orig_height}")
        
        # Calculate new dimensions based on the scale factor
        new_height = int(orig_height * scale)
        new_width = int(orig_width * scale)
        print(f"Resampling to {new_width}x{new_height}")
        
        # Read the first band and resample on the fly
        data = src.read(
            1,
            out_shape=(1, new_height, new_width),
            resampling=rasterio.enums.Resampling.nearest
        )
        # Remove the extra dimension
        data = data.squeeze()
    
    # Plot the data using a colormap (adjust cmap if needed)
    plt.figure(figsize=(10, 10))
    plt.imshow(data, cmap='viridis')
    # plt.colorbar(label='DSM value')
    plt.title(tif_path)
    plt.axis('off')
    
    # Save the figure as a PNG file
    plt.savefig(png_path, bbox_inches='tight', dpi=300)
    plt.close()
    print(f"Saved PNG: {png_path}")

# File paths for the mosaicked DSMs
tree_dsm_tif = "../data/philly/mosaic_tree_dsm.tif"
bldg_dsm_tif = "../data/philly/mosaic_building_dsm.tif"

# Output PNG file paths
tree_dsm_png = "../data/philly/mosaic_tree_dsm.png"
bldg_dsm_png = "../data/philly/mosaic_building_dsm.png"

# Visualize and export each DSM as a PNG.
visualize_and_export(tree_dsm_tif, tree_dsm_png, scale=0.1)
visualize_and_export(bldg_dsm_tif, bldg_dsm_png, scale=0.1)


Original shape for ../data/philly/mosaic_tree_dsm.tif: 90000x100000
Resampling to 9000x10000
Saved PNG: ../data/philly/mosaic_tree_dsm.png
Original shape for ../data/philly/mosaic_building_dsm.tif: 90000x100000
Resampling to 9000x10000
Saved PNG: ../data/philly/mosaic_building_dsm.png
